In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [2]:
#Step 2: Loading and Preparing the MNIST Data Set
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train)
# print(y_train)

# Convert to float32.

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)


# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
print(x_train.shape)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
print(x_train.shape)

# Normalize images value from [0, 255] to [0, 1].

x_train, x_test = x_train / 255., x_test / 255.
# print(x_train)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 784)


In [3]:
#Step 3: Setting Up Hyperparameters and Data Set Parameters
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [4]:
# STEP 4: SHUFFLING AND BATCHING THE DATA
# Use tf.data API to shuffle and batch data.

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [5]:
# STEP 5: INITIALIZING WEIGHTS AND BIASES
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.

W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
print(W)
# Bias of shape [10], the total number of classes.

b = tf.Variable(tf.zeros([num_classes]), name="bias")
print(b)

<tf.Variable 'weight:0' shape=(784, 10) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>
<tf.Variable 'bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>


In [6]:
# STEP 6: DEFINING LOGISTIC REGRESSION AND COST FUNCTION
# Logistic regression (Wx + b).

def logistic_regression(x):
    sm=tf.nn.softmax(tf.matmul(x, W) + b)
    return sm
# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.

    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.
    entropy=tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

    return entropy

In [7]:
# Accuracy metric.

def accuracy(y_pred, y_true):

    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    acc=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return acc

# Stochastic gradient descent optimizer.

optimizer = tf.optimizers.SGD(learning_rate)

In [8]:
# Optimization process. 

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, b])

  

    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, b]))

In [9]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    

    if step % display_step == 0:

        pred = logistic_regression(batch_x)

        loss = cross_entropy(pred, batch_y)

        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 486.003143, accuracy: 0.859375
step: 100, loss: 494.147644, accuracy: 0.859375
step: 150, loss: 613.357666, accuracy: 0.828125
step: 200, loss: 607.676270, accuracy: 0.839844
step: 250, loss: 443.972992, accuracy: 0.855469
step: 300, loss: 458.061279, accuracy: 0.863281
step: 350, loss: 611.057739, accuracy: 0.843750
step: 400, loss: 788.349487, accuracy: 0.800781
step: 450, loss: 341.280426, accuracy: 0.914062
step: 500, loss: 489.926178, accuracy: 0.855469
step: 550, loss: 594.245178, accuracy: 0.851562
step: 600, loss: 588.671021, accuracy: 0.839844
step: 650, loss: 637.780701, accuracy: 0.847656
step: 700, loss: 382.713623, accuracy: 0.898438
step: 750, loss: 590.901306, accuracy: 0.812500
step: 800, loss: 434.732086, accuracy: 0.875000
step: 850, loss: 623.400940, accuracy: 0.800781
step: 900, loss: 794.101440, accuracy: 0.808594
step: 950, loss: 567.593689, accuracy: 0.851562
step: 1000, loss: 419.707733, accuracy: 0.863281


In [10]:
# Test model on validation set.

pred = logistic_regression(x_test)

print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.835700
